<a href="https://colab.research.google.com/github/RyanbwChien/Fraud_data_Machine_Learning_model/blob/main/%5BRAG%5DFraud_model_upload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install langchain_community
%pip install jq
%pip install langchain_openai
%pip install qdrant_client
%pip install tiktoken
%pip install openai
%pip install faiss-cpu
%pip install scikit-learn
%pip install pymysql
%pip install python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.6/746.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.7/306.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 20.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.6 MB/s eta 0:00:00


TypeError: str expected, not NoneType

In [2]:
import os
import dotenv
import pymysql
import pandas as pd
import numpy as np
# GCP SQL database connection
dotenv.load_dotenv("/content/drive/MyDrive/.env")

db_config = {
    "database": "Anti_Fraud",
    "charset": "utf8mb4",
    "host": os.getenv('DB_HOST', 'localhost'),
    "user": os.getenv('DB_USER', 'root'),
    "password": os.getenv('DB_PASSWORD', 'password')

}

conn = pymysql.connect(**db_config)
cursor = conn.cursor()

query = "SELECT * FROM Case_processing;"
cursor.execute(query)
result = cursor.fetchall()
columns = [col[0] for col in cursor.description]

Fraud_data = pd.DataFrame(result,columns=columns)
Fraud_data = Fraud_data.replace({None: np.nan})
# pd.read_sql_query(query, conn)
len(Fraud_data)

50356

In [3]:
# 1. 讀取 JSON 檔案
import json
import re
def preprocess_text(text):
    text = re.sub(r"<br\s*/?>", ". ", text)
    text = re.sub(r"\s+", " ", text.strip())
    return text

json_file = r"/content/drive/MyDrive/linechatbox_mysql_V1.json"
with open(json_file, "r", encoding="utf-8") as f:
    json_data = json.load(f)

# 2. 提取 JSON 內容
documents = [preprocess_text(item["Content"]) for item in json_data if "Content" in item]
Title = [preprocess_text(item["Title"]) for item in json_data if "Title" in item]
Reported_Date = [item["Reported_Date"] for item in json_data if "Reported_Date" in item]

# 3. 轉換為 LangChain Document 格式
from langchain.docstore.document import Document
doc_objects = [Document(page_content=text[0:1400],metadata={
            "Title": title,  # 原始文本長度
            "Reported_Date": date
        }) for text,title,date in zip(documents,Title,Reported_Date)]

# 4. 使用 RecursiveCharacterTextSplitter 切分文本
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(doc_objects)

In [ ]:
split_documents

[Document(metadata={'Title': '[問題] 求救！客戶正在被詐騙中，無法清醒', 'Reported_Date': '2023-06-06'}, page_content='作者Charmed (酒神精靈。美神亮膚。)看板Bunco標題[問題] 求救！客戶正在被詐騙中，無法清醒時間Tue Jun 6 00:42:42 2023 《求救文》 我本身是做一個銀行貸款的代辦業者 我做這一行10幾年了 這十幾年來有一個單親媽媽跟我特別要好 一個人要養兩個孩子很辛苦 這十年都是靠著我幫她貸款 和她辛苦工作養大這兩個小孩的， 所以我跟她之間有革命情感 上星期她打電話來說 她急需要30萬， 問他用途，他也都不講 這是他第一次在我面前不講 過了兩天又說不想辦了 因為已經跟親戚借到 了， 我說你不會是找別人辦了吧 她說她姪女借她30萬了 今天傍晚我要下班時 突然接到她的電話， 居然問我能不能私人借他30？ 因為他這筆錢 明天就要用到了 我說到底是什麼用途？ （但他因為她的行為舉止就是很奇怪） 後來在我逼問下 她才老實說有人帶她在網站平台玩澳門彩票 前兩天他已經把子女借她的30匯給對方了 今天人家要把本金和獎金 匯給她的時候發現寫錯帳號 現在要更正這些程序要一筆30萬元的押金跑流程， 她還一直說我明天就還你了 因為明天這筆錢就下來了 我簡直快暈倒了 我一直告訴她 妳明天千萬不要再匯這30 我告訴她這是詐騙 這30肉包子打狗有去無回 可是她現在一頭熱 不相信自己被騙的事實 叫我幫她想辦法想辦法 明天就要用到了 我怎麼勸她都聽不進去 剛剛還傳Line來問我有幫我問到消息了嗎？ 有金主或代書借她嗎？ 天啊，我該怎麼樣讓他清醒！！ ！ PS 剛剛有傳Line來，又要把金額追加到40萬'),
 Document(metadata={'Title': '[親身] 今天剛發生的', 'Reported_Date': '2008-07-11'}, page_content='作者pa7711201 (粽子優格)看板Bunco標題[親身] 今天剛發生的時間Fri Jul 11 19:45:27 2008 昨天閃光接到拍賣賣家的電話說匯款的帳戶被弄成分期付款的方式匯款 要她去郵局檢查(但是我閃光明明是現金匯款 而且是一次付清) 結果後來又有自稱是郵局行員的打電話給閃光叫她把錢全部

In [ ]:
length = []
for i in doc_objects:
  length.append(len(i.page_content))

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings
import openai
import os

# 載入 Sentence Embedding
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-zh", model_kwargs={"device": "cuda"})
# 使用 FAISS 創建向量資料庫
from langchain.vectorstores import FAISS
db = FAISS.from_documents(split_documents, embeddings)

<ipython-input-3-8da7ad21990b>:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-zh", model_kwargs={"device": "cuda"})
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/27.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
db.save_local("/content/drive/MyDrive/faiss_index_dir2")

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-zh", model_kwargs={"device": "cpu"})
# 重新載入 FAISS
db = FAISS.load_local("/content/drive/MyDrive/faiss_index_dir2", embeddings,allow_dangerous_deserialization=True)

In [ ]:
# 定義查詢
query = """IG有哪些詐騙手法"""
# retrieved_docs = db.similarity_search(query, k=10)
# retrieved_docs = db.similarity_search_with_relevance_scores(query, k=10, score_threshold=0.5)
retrieved_docs = db.max_marginal_relevance_search(query, k=10, fetch_k=20, lambda_mult=0.7)
# 打印输出检索到的文档
print("检索到的文档：")
for doc in retrieved_docs:
    print(doc)


检索到的文档：
page_content='我接獲IG訊息，對方假冒我的親友向我誆稱急需用錢，我與對方聯繫後，對方請我匯款至他指定帳戶，後續又要求匯款第二次，我覺得有異所以立即打電話給朋友查證，才驚覺遭詐騙。' metadata={'Title': '盜用網路帳號詐騙', 'Reported_Date': '2024-12-17'}
page_content='近期詐騙集團大量創設IG社群帳號，佯稱舉辦抽獎活動，標的物包含3C商品、現金、名牌衣飾精品、塔羅占卜算命及婦嬰用品等，吸引民眾留言參加。 後詐騙集團私訊被害人謊稱中獎，再以會員費、代購費、核實帳戶、保證金、可再次抽獎等話術，要求被害人提供個人資料及陸續匯款至被害人因匯款金額過大、未收到退款等狀況始驚覺遭詐。 要領獎須先匯款? 小心是詐騙!' metadata={'Title': '領獎前得先匯款？IG充斥抽獎詐騙!', 'Reported_Date': '2024-05-31'}
page_content='我和朋友介紹之客人得知投資訊息，歹徒慫恿至【網站名稱：IG及網址：https:ig.markesin.com】，誆稱保證獲利、穩賺不賠，我依指示至該網站申請帳號，並依照歹徒指示購買虛擬貨幣，惟因跟朋友提到後，朋友警告這是詐騙，驚覺受騙，計損失新臺幣4481438元。' metadata={'Title': '假交友(投資詐財)詐騙', 'Reported_Date': '2025-01-16'}
page_content='我在ig看見一則動態廣告抽獎活動，之後點擊廣告後就自動進入對方提供的line好友進行對話，line向我稱須依照其指示方能得到抽獎的指定金額，我不疑有他依指示操作後，使用網路轉帳至對方提供帳戶，共損失新臺幣32241元，後續對方又表示需付款核實支付認證費用，我驚覺不對勁為詐騙而報案。' metadata={'Title': '假中獎通知詐騙', 'Reported_Date': '2024-12-22'}
page_content='我於11日下午2.49分被詐騙轉帳，一開始IG抽獎以為中獎 後來說匯款不成功要解除需要先匯款出來再一次轉入，結果被詐騙。' metadata={'Title': '中獎通知詐騙', 'Reported_Date': '2025-01-01'}
page_co

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

system_template = """你是一名人工智慧助理，專職於反詐騙服務領域，
使用以下上下文請簡短回答用戶的問題並控制字數在100以內：
----------------
{context}"""
human_template = "{question}"

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template(human_template),
]

qa_prompt = ChatPromptTemplate.from_messages(messages)
retriever = db.as_retriever(search_kwargs={'k': 10})

# 使用 OpenAI Chat 模型
llm = ChatOpenAI(model="gpt-4o", api_key=os.environ["OPENAI_API_KEY"])
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=False,
    chain_type_kwargs={"verbose": False,"prompt": qa_prompt})

<ipython-input-27-77005a0476a6>:29: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o", api_key=os.environ["OPENAI_API_KEY"])


In [ ]:
# 定義運行問答的函數
def run_my_rag(qa, query):
    print(f"Query: {query}\n")
    result = qa.run(query)
    print("\nResult: ", result)
### Ask Queries Now
# 定義查詢
query = """IG有哪些詐騙手法"""
# 執行問答
run_my_rag(qa, query)

Query: IG有哪些詐騙手法


Result:  在IG上常見的詐騙手法包括：

1. 假冒親友：假裝是你的親友，要求匯款幫助。
2. 投資詐騙：誘導你參加虛假的投資計畫，承諾高回報。
3. 抽獎活動：宣稱中獎但需支付手續費或認證費。
4. 假冒客服：以購物或中獎名義要求提供個人或銀行資訊。

務必提高警覺，勿輕信陌生訊息。
